# Machine Learning Pipelines: Complete Preprocessing & Model Evaluation

## Overview
This notebook demonstrates a complete machine learning pipeline from raw data to model evaluation. We'll use the Tips dataset to predict meal timing (Lunch vs Dinner) based on customer and meal characteristics.

## Learning Objectives
By the end of this notebook, you'll understand:
- Data loading and exploratory data analysis
- Target variable encoding for categorical labels
- Train-test splitting with stratification
- Missing value analysis and handling
- Preprocessing pipeline construction
- Column transformers for mixed data types
- Model training and hyperparameter optimization
- Performance evaluation and comparison

## Technical Stack
- **pandas**: Data manipulation and analysis
- **scikit-learn**: Machine learning algorithms and preprocessing
- **seaborn**: Dataset loading and visualization
- **numpy**: Numerical computing

## Dataset: Restaurant Tips
- **Source**: Seaborn built-in dataset
- **Samples**: 244 restaurant visits
- **Features**: Bill amount, tip, party size, customer demographics
- **Target**: Meal time (Lunch vs Dinner)
- **Task**: Binary classification

## Pipeline Workflow
```
Raw Data → EDA → Target Encoding → Train/Test Split → 
Missing Value Check → Feature Pipeline Design → 
Data Preprocessing → Model Training → Hyperparameter Tuning → Evaluation
```

In [ ]:
# Import Essential Libraries
# Core data manipulation and analysis
import pandas as pd          # DataFrame operations and data manipulation
import numpy as np           # Numerical computing and array operations
import matplotlib.pyplot as plt  # Basic plotting and visualization
import seaborn as sns        # Statistical visualizations and built-in datasets

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

print("Libraries imported successfully.")
print("Ready for data loading, EDA, preprocessing, and model training.")

✅ All libraries imported successfully!
🎯 Ready for: Data loading, EDA, preprocessing, and model training


In [ ]:
# Load the Tips Dataset
# Load the 'tips' dataset from seaborn
# This dataset contains information about restaurant tips, meals, and customer demographics
df = sns.load_dataset("tips")

print("Dataset loaded successfully.")
print(f"Dataset shape: {df.shape[0]} rows × {df.shape[1]} columns")
print("\nFirst few rows:")
df.head()

📊 DATASET LOADED SUCCESSFULLY!
Dataset shape: 244 rows × 7 columns

🔍 First few rows:


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
# Define Prediction Target
# We'll predict 'time' - whether the meal was during Lunch or Dinner
print("Prediction objective: Classify meal time (Lunch vs Dinner)")
print("=" * 55)

# Explore unique values in the target variable
unique_times = df.time.unique()
print(f"Unique time periods: {unique_times}")
print(f"Number of classes: {len(unique_times)}")

# Count distribution
time_counts = df.time.value_counts()
print(f"\nClass distribution:")
for time_period, count in time_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   {time_period}: {count} samples ({percentage:.1f}%)")
    
print(f"\nThis is a binary classification problem.")
print(f"We'll use customer features to predict meal timing.")

🎯 PREDICTION OBJECTIVE: Classify meal time (Lunch vs Dinner)
📋 Unique time periods: ['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']
📊 Number of classes: 2

📈 Class distribution:
   Dinner: 176 samples (72.1%)
   Lunch: 68 samples (27.9%)

💡 This is a binary classification problem!
   We'll use customer features to predict meal timing


In [7]:
# Exploratory Data Analysis (EDA)
print("Dataset Overview:")
print("=" * 20)

# Get dataset information
print("Dataset Info:")
print(f"   Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"   Memory usage: {df.memory_usage(deep=True).sum()} bytes")
print()

# Display data types and non-null counts
print("Column Details:")
df.info()

print(f"\nSummary Statistics:")
print("=" * 18)
df.describe()

🕵️ DATASET OVERVIEW:
📋 Dataset Info:
   • Shape: 244 rows × 7 columns
   • Memory usage: 8069 bytes

🏗️ Column Details:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB

📊 SUMMARY STATISTICS:


,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [8]:
# Target Variable Encoding
# Since 'time' is categorical (nominal), we use LabelEncoder
# This converts text labels to numerical values for ML algorithms
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
print(f"Original values: {df['time'].unique()}")

# Transform categorical time values to numerical
df['time'] = encoder.fit_transform(df['time'])

print(f"Encoded values: {df['time'].unique()}")
print(f"Encoding mapping:")
for i, label in enumerate(encoder.classes_):
    print(f"   '{label}' → {i}")

print(f"\nTarget variable successfully encoded.")
print(f"Class distribution after encoding:")
print(df['time'].value_counts().sort_index())

# Display the updated dataset
print(f"\nUpdated dataset:")
df

🏷️ ENCODING TARGET VARIABLE (TIME)
📝 Original values: ['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']
🔢 Encoded values: [0 1]
📊 Encoding mapping:
   'Dinner' → 0
   'Lunch' → 1

✅ Target variable successfully encoded!
📈 Class distribution after encoding:
time
0    176
1     68
Name: count, dtype: int64

🔍 Updated dataset:


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [9]:
# ✅ VERIFICATION OF TARGET ENCODING
# ==================================

print("🔍 VERIFYING TARGET VARIABLE ENCODING:")
print("=" * 42)

# Verify the encoding worked correctly
encoded_values = df.time.unique()
print(f"📊 Encoded time values: {sorted(encoded_values)}")
print(f"📈 Value counts:")

for value in sorted(encoded_values):
    count = (df.time == value).sum()
    original_label = encoder.inverse_transform([value])[0]
    print(f"   {value} ('{original_label}'): {count} samples")

print(f"\n✅ Target encoding verification complete!")
print(f"💡 Ready for train-test split and model training")

🔍 VERIFYING TARGET VARIABLE ENCODING:
📊 Encoded time values: [np.int64(0), np.int64(1)]
📈 Value counts:
   0 ('Dinner'): 176 samples
   1 ('Lunch'): 68 samples

✅ Target encoding verification complete!
💡 Ready for train-test split and model training


In [10]:
# 🎯 PREPARING FEATURES AND TARGET VARIABLES
# ===========================================

print("🎯 SEPARATING FEATURES AND TARGET:")
print("=" * 38)

# Separate features (X) and target variable (y)
X = df.drop('time', axis=1)  # Features: all columns except 'time'
y = df.time                  # Target: 'time' column (0=Dinner, 1=Lunch)

print(f"📊 DATASET SPLIT SUMMARY:")
print(f"   Features (X): {X.shape[0]} samples × {X.shape[1]} features")
print(f"   Target (y):   {y.shape[0]} samples")
print()

print(f"📋 FEATURE COLUMNS:")
print(f"   {list(X.columns)}")
print()

print(f"🎯 TARGET DISTRIBUTION:")
for value in sorted(y.unique()):
    count = (y == value).sum()
    original_label = encoder.inverse_transform([value])[0]
    percentage = (count / len(y)) * 100
    print(f"   {value} ('{original_label}'): {count} samples ({percentage:.1f}%)")

print(f"\n✅ Features and target successfully separated!")
print(f"💡 Ready for train-test split")

🎯 SEPARATING FEATURES AND TARGET:
📊 DATASET SPLIT SUMMARY:
   Features (X): 244 samples × 6 features
   Target (y):   244 samples

📋 FEATURE COLUMNS:
   ['total_bill', 'tip', 'sex', 'smoker', 'day', 'size']

🎯 TARGET DISTRIBUTION:
   0 ('Dinner'): 176 samples (72.1%)
   1 ('Lunch'): 68 samples (27.9%)

✅ Features and target successfully separated!
💡 Ready for train-test split


In [11]:
# 🔄 TRAIN-TEST SPLIT
# ===================

print("🔄 CREATING TRAIN-TEST SPLIT:")
print("=" * 32)

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split the data: 80% training, 20% testing
# random_state=1 ensures reproducible results
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% for testing
    random_state=1,     # For reproducibility
    stratify=y          # Maintain class distribution in both splits
)

print(f"📊 SPLIT SUMMARY:")
print(f"   Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   Test set:     {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print()

print(f"🎯 CLASS DISTRIBUTION CHECK:")
print(f"   Training target distribution:")
for value in sorted(y_train.unique()):
    count = (y_train == value).sum()
    percentage = (count / len(y_train)) * 100
    original_label = encoder.inverse_transform([value])[0]
    print(f"     {value} ('{original_label}'): {count} samples ({percentage:.1f}%)")

print(f"   Test target distribution:")
for value in sorted(y_test.unique()):
    count = (y_test == value).sum()
    percentage = (count / len(y_test)) * 100
    original_label = encoder.inverse_transform([value])[0]
    print(f"     {value} ('{original_label}'): {count} samples ({percentage:.1f}%)")

print(f"\n✅ Train-test split completed successfully!")
print(f"💡 Stratified split maintains balanced class distribution")

🔄 CREATING TRAIN-TEST SPLIT:
📊 SPLIT SUMMARY:
   Training set: 195 samples (79.9%)
   Test set:     49 samples (20.1%)

🎯 CLASS DISTRIBUTION CHECK:
   Training target distribution:
     0 ('Dinner'): 141 samples (72.3%)
     1 ('Lunch'): 54 samples (27.7%)
   Test target distribution:
     0 ('Dinner'): 35 samples (71.4%)
     1 ('Lunch'): 14 samples (28.6%)

✅ Train-test split completed successfully!
💡 Stratified split maintains balanced class distribution


In [12]:
# 🔍 EXAMINING TRAINING DATA STRUCTURE
# ====================================

print("🔍 TRAINING DATA EXAMINATION:")
print("=" * 33)

# Display first few rows of training features
print(f"📊 Training set shape: {X_train.shape}")
print(f"📋 Feature columns: {list(X_train.columns)}")
print()

print(f"🔍 First 5 training samples:")
display(X_train.head())

print(f"\n📈 DATA TYPES:")
for col in X_train.columns:
    dtype = X_train[col].dtype
    unique_count = X_train[col].nunique()
    print(f"   {col:<12}: {dtype} ({unique_count} unique values)")

print(f"\n💡 OBSERVATIONS:")
print(f"   • Numerical features: total_bill, tip, size")  
print(f"   • Categorical features: sex, smoker, day")
print(f"   • Mixed data types require preprocessing pipeline!")
print(f"   • Next: Check for missing values and apply preprocessing")

🔍 TRAINING DATA EXAMINATION:
📊 Training set shape: (195, 6)
📋 Feature columns: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'size']

🔍 First 5 training samples:


,total_bill,tip,sex,smoker,day,size
89,21.16,3.00,Male,No,Thur,2
11,35.26,5.00,Female,No,Sun,4
227,20.45,3.00,Male,No,Sat,4
86,13.03,2.00,Male,No,Thur,2
85,34.83,5.17,Female,No,Thur,4



📈 DATA TYPES:
   total_bill  : float64 (189 unique values)
   tip         : float64 (106 unique values)
   sex         : category (2 unique values)
   smoker      : category (2 unique values)
   day         : category (4 unique values)
   size        : int64 (6 unique values)

💡 OBSERVATIONS:
   • Numerical features: total_bill, tip, size
   • Categorical features: sex, smoker, day
   • Mixed data types require preprocessing pipeline!
   • Next: Check for missing values and apply preprocessing


In [13]:
# 🔍 MISSING VALUE ANALYSIS
# =========================

print("🔍 MISSING VALUE ANALYSIS:")
print("=" * 29)

# Check for missing values in the dataset
missing_values = df.isna().sum()
total_samples = len(df)

print(f"📊 Missing Value Report:")
print(f"   Total samples: {total_samples}")
print()

if missing_values.sum() == 0:
    print("✅ EXCELLENT! No missing values found in any column!")
    print("   This simplifies our preprocessing pipeline")
else:
    print("⚠️  Missing values detected:")
    for column, missing_count in missing_values.items():
        if missing_count > 0:
            percentage = (missing_count / total_samples) * 100
            print(f"   {column}: {missing_count} missing ({percentage:.2f}%)")

print(f"\n📋 Column-wise missing value summary:")
missing_summary = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Missing_Percentage': (missing_values.values / total_samples * 100).round(2)
})
display(missing_summary)

print(f"\n💡 PREPROCESSING STRATEGY:")
if missing_values.sum() == 0:
    print("   • No missing value imputation needed")
    print("   • Focus on encoding categorical variables")
    print("   • Apply feature scaling to numerical variables")
else:
    print("   • Will handle missing values in preprocessing pipeline")
    print("   • Use median imputation for numerical features")
    print("   • Use most frequent imputation for categorical features")

🔍 MISSING VALUE ANALYSIS:
📊 Missing Value Report:
   Total samples: 244

✅ EXCELLENT! No missing values found in any column!
   This simplifies our preprocessing pipeline

📋 Column-wise missing value summary:


,Column,Missing_Count,Missing_Percentage
0,total_bill,0,0.0
1,tip,0,0.0
2,sex,0,0.0
3,smoker,0,0.0
4,day,0,0.0
5,time,0,0.0
6,size,0,0.0



💡 PREPROCESSING STRATEGY:
   • No missing value imputation needed
   • Focus on encoding categorical variables
   • Apply feature scaling to numerical variables


In [14]:
# 🛠️ IMPORTING PREPROCESSING TOOLS
# =================================

print("🛠️ IMPORTING PREPROCESSING COMPONENTS:")
print("=" * 42)

# Missing value handling
from sklearn.impute import SimpleImputer
print("✅ SimpleImputer: Handle missing values")

# Feature encoding and scaling
from sklearn.preprocessing import OneHotEncoder
print("✅ OneHotEncoder: Convert categorical to binary features")

from sklearn.preprocessing import StandardScaler  
print("✅ StandardScaler: Normalize numerical features")

# Pipeline construction tools
from sklearn.pipeline import Pipeline
print("✅ Pipeline: Chain preprocessing steps")

from sklearn.compose import ColumnTransformer
print("✅ ColumnTransformer: Apply different preprocessing to different columns")

print(f"\n📋 PREPROCESSING TOOLKIT READY:")
print(f"   🔧 Missing Values → SimpleImputer")
print(f"   🔧 Categorical Data → OneHotEncoder") 
print(f"   🔧 Numerical Data → StandardScaler")
print(f"   🔧 Pipeline Management → Pipeline & ColumnTransformer")

print(f"\n💡 NEXT STEPS:")
print(f"   1. Define numerical and categorical columns")
print(f"   2. Create separate pipelines for each data type")
print(f"   3. Combine pipelines using ColumnTransformer")
print(f"   4. Apply preprocessing to training and test data")

🛠️ IMPORTING PREPROCESSING COMPONENTS:
✅ SimpleImputer: Handle missing values
✅ OneHotEncoder: Convert categorical to binary features
✅ StandardScaler: Normalize numerical features
✅ Pipeline: Chain preprocessing steps
✅ ColumnTransformer: Apply different preprocessing to different columns

📋 PREPROCESSING TOOLKIT READY:
   🔧 Missing Values → SimpleImputer
   🔧 Categorical Data → OneHotEncoder
   🔧 Numerical Data → StandardScaler
   🔧 Pipeline Management → Pipeline & ColumnTransformer

💡 NEXT STEPS:
   1. Define numerical and categorical columns
   2. Create separate pipelines for each data type
   3. Combine pipelines using ColumnTransformer
   4. Apply preprocessing to training and test data


In [15]:
# 📊 DEFINING COLUMN TYPES FOR PREPROCESSING
# ==========================================

print("📊 CATEGORIZING FEATURES BY DATA TYPE:")
print("=" * 41)

# Define categorical columns (non-numerical features)
cat_cols = ["sex", "smoker", "day"]
print(f"🏷️  CATEGORICAL COLUMNS: {cat_cols}")
print(f"   • These contain text/category values")
print(f"   • Need OneHotEncoder to convert to numbers")
print(f"   • Will expand into multiple binary columns")

# Define numerical columns (already numerical features)  
num_col = ["total_bill", "tip", "size"]
print(f"\n🔢 NUMERICAL COLUMNS: {num_col}")
print(f"   • These are already numeric values")
print(f"   • Need StandardScaler for normalization")
print(f"   • Will maintain same number of columns")

print(f"\n📋 PREPROCESSING REQUIREMENTS:")
print(f"   Categorical ({len(cat_cols)} cols) → OneHot Encoding")
print(f"   Numerical ({len(num_col)} cols)   → Standard Scaling")

print(f"\n💡 EXPECTED TRANSFORMATION:")
print(f"   Original features: {len(cat_cols) + len(num_col)} columns")
print(f"   After preprocessing: {len(num_col)} + (binary features from encoding)")
print(f"   Final feature count will be determined by unique categorical values")

📊 CATEGORIZING FEATURES BY DATA TYPE:
🏷️  CATEGORICAL COLUMNS: ['sex', 'smoker', 'day']
   • These contain text/category values
   • Need OneHotEncoder to convert to numbers
   • Will expand into multiple binary columns

🔢 NUMERICAL COLUMNS: ['total_bill', 'tip', 'size']
   • These are already numeric values
   • Need StandardScaler for normalization
   • Will maintain same number of columns

📋 PREPROCESSING REQUIREMENTS:
   Categorical (3 cols) → OneHot Encoding
   Numerical (3 cols)   → Standard Scaling

💡 EXPECTED TRANSFORMATION:
   Original features: 6 columns
   After preprocessing: 3 + (binary features from encoding)
   Final feature count will be determined by unique categorical values


In [16]:
# 🏗️ BUILDING PREPROCESSING PIPELINES
# ====================================

print("🏗️ CONSTRUCTING FEATURE PREPROCESSING PIPELINES:")
print("=" * 49)

# NUMERICAL PIPELINE: Handle numerical features
print("🔢 NUMERICAL PIPELINE:")
num_pipeline = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy="median")),  # Fill missing with median
    ('scaling', StandardScaler())                      # Normalize to mean=0, std=1
])
print("   Step 1: Impute missing values with median")
print("   Step 2: Standardize features (mean=0, std=1)")

# CATEGORICAL PIPELINE: Handle categorical features  
print(f"\n🏷️  CATEGORICAL PIPELINE:")
cat_pipeline = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy="most_frequent")),  # Fill missing with mode
    ('encoding', OneHotEncoder())                            # Convert to binary features
])
print("   Step 1: Impute missing values with most frequent value")
print("   Step 2: One-hot encode categories into binary columns")

print(f"\n✅ PIPELINE CONSTRUCTION COMPLETE!")
print(f"\n📋 PIPELINE SUMMARY:")
print(f"   • Numerical Pipeline: {len(num_pipeline.steps)} steps")
print(f"   • Categorical Pipeline: {len(cat_pipeline.steps)} steps")
print(f"   • Both pipelines handle missing values + transformations")

print(f"\n💡 WHAT EACH PIPELINE DOES:")
print(f"   📈 Numerical: missing → median, then scale to standard normal")
print(f"   🔤 Categorical: missing → most frequent, then binary encoding") 
print(f"   🎯 Result: All features become numerical for ML algorithms")

🏗️ CONSTRUCTING FEATURE PREPROCESSING PIPELINES:
🔢 NUMERICAL PIPELINE:
   Step 1: Impute missing values with median
   Step 2: Standardize features (mean=0, std=1)

🏷️  CATEGORICAL PIPELINE:
   Step 1: Impute missing values with most frequent value
   Step 2: One-hot encode categories into binary columns

✅ PIPELINE CONSTRUCTION COMPLETE!

📋 PIPELINE SUMMARY:
   • Numerical Pipeline: 2 steps
   • Categorical Pipeline: 2 steps
   • Both pipelines handle missing values + transformations

💡 WHAT EACH PIPELINE DOES:
   📈 Numerical: missing → median, then scale to standard normal
   🔤 Categorical: missing → most frequent, then binary encoding
   🎯 Result: All features become numerical for ML algorithms


In [17]:
# 🔗 COMBINING PIPELINES WITH COLUMNTRANSFORMER
# =============================================

print("🔗 CREATING UNIFIED PREPROCESSING PIPELINE:")
print("=" * 44)

# Combine both pipelines using ColumnTransformer
# This applies different preprocessing to different column types
preprocessor = ColumnTransformer([
    ("num_pipeline", num_pipeline, num_col),    # Apply num_pipeline to numerical columns
    ("cat_pipeline", cat_pipeline, cat_cols)    # Apply cat_pipeline to categorical columns
])

print("✅ UNIFIED PREPROCESSOR CREATED!")
print(f"\n📊 PREPROCESSING CONFIGURATION:")
print(f"   🔢 Numerical columns ({len(num_col)}): {num_col}")
print(f"      → Pipeline: median imputation + standard scaling")
print(f"   🏷️  Categorical columns ({len(cat_cols)}): {cat_cols}")  
print(f"      → Pipeline: mode imputation + one-hot encoding")

print(f"\n🎯 TRANSFORMATION WORKFLOW:")
print(f"   1. Input: Raw data with mixed types")
print(f"   2. Split: Route columns to appropriate pipelines")
print(f"   3. Process: Apply pipeline transformations")
print(f"   4. Combine: Merge processed features")
print(f"   5. Output: Fully numerical feature matrix")

print(f"\n💡 EXPECTED OUTPUT STRUCTURE:")
print(f"   • Numerical features: {len(num_col)} columns (scaled)")
print(f"   • Categorical features: Multiple binary columns (one-hot)")
print(f"   • Total: More columns than original due to encoding")
print(f"   • All values: Numerical and ready for ML algorithms!")

print(f"\n🚀 PREPROCESSOR READY FOR TRAINING DATA!")

🔗 CREATING UNIFIED PREPROCESSING PIPELINE:
✅ UNIFIED PREPROCESSOR CREATED!

📊 PREPROCESSING CONFIGURATION:
   🔢 Numerical columns (3): ['total_bill', 'tip', 'size']
      → Pipeline: median imputation + standard scaling
   🏷️  Categorical columns (3): ['sex', 'smoker', 'day']
      → Pipeline: mode imputation + one-hot encoding

🎯 TRANSFORMATION WORKFLOW:
   1. Input: Raw data with mixed types
   2. Split: Route columns to appropriate pipelines
   3. Process: Apply pipeline transformations
   4. Combine: Merge processed features
   5. Output: Fully numerical feature matrix

💡 EXPECTED OUTPUT STRUCTURE:
   • Numerical features: 3 columns (scaled)
   • Categorical features: Multiple binary columns (one-hot)
   • Total: More columns than original due to encoding
   • All values: Numerical and ready for ML algorithms!

🚀 PREPROCESSOR READY FOR TRAINING DATA!


In [18]:
# 🔄 TRANSFORM TRAINING DATA
# =========================

# Apply preprocessing pipeline to training data
print("🔄 Applying preprocessing to training data...")
X_train_transformed = preprocessor.fit_transform(X_train)

print("✅ Training data transformation complete!")
print(f"Original training shape: {X_train.shape}")
print(f"Transformed training shape: {X_train_transformed.shape}")
print(f"Features expanded from {X_train.shape[1]} to {X_train_transformed.shape[1]} due to one-hot encoding")
print()
print("💡 What happened:")
print("   • Categorical variables (sex, smoker, day) converted to binary features")
print("   • Numerical variables scaled to standard normal distribution")
print("   • Missing values imputed (if any)")
print("   • Data is now ready for machine learning models!")

🔄 Applying preprocessing to training data...
✅ Training data transformation complete!
Original training shape: (195, 6)
Transformed training shape: (195, 11)
Features expanded from 6 to 11 due to one-hot encoding

💡 What happened:
   • Categorical variables (sex, smoker, day) converted to binary features
   • Numerical variables scaled to standard normal distribution
   • Missing values imputed (if any)
   • Data is now ready for machine learning models!


In [19]:
# 🔄 TRANSFORM TEST DATA
# ======================

# Apply preprocessing pipeline to test data (using already fitted preprocessor)
print("🔄 Applying preprocessing to test data...")
X_test_transformed = preprocessor.transform(X_test)

print("✅ Test data transformation complete!")
print(f"Original test shape: {X_test.shape}")
print(f"Transformed test shape: {X_test_transformed.shape}")
print()
print("💡 Important notes:")
print("   • Used transform() not fit_transform() to avoid data leakage")
print("   • Same preprocessing applied as learned from training data")
print("   • Both datasets now have consistent feature structure")
print("   • Ready for model training and evaluation!")

🔄 Applying preprocessing to test data...
✅ Test data transformation complete!
Original test shape: (49, 6)
Transformed test shape: (49, 11)

💡 Important notes:
   • Used transform() not fit_transform() to avoid data leakage
   • Same preprocessing applied as learned from training data
   • Both datasets now have consistent feature structure
   • Ready for model training and evaluation!


In [20]:
# 👀 EXAMINING RAW TRAINING DATA (BEFORE PREPROCESSING)
# ====================================================

print("👀 RAW TRAINING DATA INSPECTION:")
print("=" * 36)

print(f"📊 Training data shape: {X_train.shape}")
print(f"📋 Data types before preprocessing:")

for col in X_train.columns:
    dtype = X_train[col].dtype
    unique_vals = X_train[col].nunique()
    sample_vals = X_train[col].unique()[:3]  # Show first 3 unique values
    print(f"   {col:<12}: {dtype} | {unique_vals} unique | sample: {sample_vals}")

print(f"\n🔍 Raw training data (first 5 rows):")
display(X_train)

print(f"\n⚠️  NOTICE: This data contains mixed types!")
print(f"   • Categorical columns have string values ('Male', 'Female', etc.)")
print(f"   • Numerical columns have float/int values")
print(f"   • ML algorithms can't handle strings directly")
print(f"   • This is WHY we need preprocessing!")

print(f"\n🔄 NEXT: Apply preprocessor to convert everything to numbers")

👀 RAW TRAINING DATA INSPECTION:
📊 Training data shape: (195, 6)
📋 Data types before preprocessing:
   total_bill  : float64 | 189 unique | sample: [21.16 35.26 20.45]
   tip         : float64 | 106 unique | sample: [3. 5. 2.]
   sex         : category | 2 unique | sample: ['Male', 'Female']
Categories (2, object): ['Male', 'Female']
   smoker      : category | 2 unique | sample: ['No', 'Yes']
Categories (2, object): ['Yes', 'No']
   day         : category | 4 unique | sample: ['Thur', 'Sun', 'Sat']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']
   size        : int64 | 6 unique | sample: [2 4 3]

🔍 Raw training data (first 5 rows):


,total_bill,tip,sex,smoker,day,size
89,21.16,3.00,Male,No,Thur,2
11,35.26,5.00,Female,No,Sun,4
227,20.45,3.00,Male,No,Sat,4
86,13.03,2.00,Male,No,Thur,2
85,34.83,5.17,Female,No,Thur,4
...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,6
160,21.50,3.50,Male,No,Sun,4
40,16.04,2.24,Male,No,Sat,3
82,10.07,1.83,Female,No,Thur,1



⚠️  NOTICE: This data contains mixed types!
   • Categorical columns have string values ('Male', 'Female', etc.)
   • Numerical columns have float/int values
   • ML algorithms can't handle strings directly
   • This is WHY we need preprocessing!

🔄 NEXT: Apply preprocessor to convert everything to numbers


In [21]:
# 🤖 IMPORTING MACHINE LEARNING MODELS
# ====================================

print("🤖 SETTING UP MACHINE LEARNING MODELS:")
print("=" * 40)

# Import classification algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

print("✅ Imported classification algorithms:")
print("   🌳 DecisionTreeClassifier: Tree-based learning")
print("   🎯 SVC (Support Vector Classifier): Margin-based learning")

# Create model dictionary for easy iteration
models = {
    "Support Vector Classifier": SVC(),
    "Decision Tree Classifier": DecisionTreeClassifier()
}

print(f"\n📊 MODEL COLLECTION CREATED:")
print(f"   Total models: {len(models)}")
for name, model in models.items():
    print(f"   • {name}: {type(model).__name__}")

print(f"\n🎯 MODEL CHARACTERISTICS:")
print(f"   📈 SVC: Finds optimal decision boundary with maximum margin")
print(f"   🌲 Decision Tree: Creates if-then rules for classification")
print(f"   🔄 Both: Will be trained and evaluated on preprocessed data")

print(f"\n💡 EVALUATION STRATEGY:")
print(f"   • Train both models on same preprocessed training data")
print(f"   • Test both models on same preprocessed test data") 
print(f"   • Compare accuracy scores to find best performer")
print(f"   • Use standardized evaluation function for consistency")

print(f"\n🚀 MODELS READY FOR TRAINING!")

🤖 SETTING UP MACHINE LEARNING MODELS:
✅ Imported classification algorithms:
   🌳 DecisionTreeClassifier: Tree-based learning
   🎯 SVC (Support Vector Classifier): Margin-based learning

📊 MODEL COLLECTION CREATED:
   Total models: 2
   • Support Vector Classifier: SVC
   • Decision Tree Classifier: DecisionTreeClassifier

🎯 MODEL CHARACTERISTICS:
   📈 SVC: Finds optimal decision boundary with maximum margin
   🌲 Decision Tree: Creates if-then rules for classification
   🔄 Both: Will be trained and evaluated on preprocessed data

💡 EVALUATION STRATEGY:
   • Train both models on same preprocessed training data
   • Test both models on same preprocessed test data
   • Compare accuracy scores to find best performer
   • Use standardized evaluation function for consistency

🚀 MODELS READY FOR TRAINING!


In [22]:
# 📏 MODEL TRAINING AND EVALUATION FUNCTION
# =========================================

print("📏 CREATING MODEL EVALUATION FUNCTION:")
print("=" * 40)

from sklearn.metrics import accuracy_score

def model_train_eval(X_train, y_train, X_test, y_test, models):
    """
    🎯 COMPREHENSIVE MODEL TRAINING & EVALUATION
    
    This function:
    1. Trains each model on training data
    2. Makes predictions on test data  
    3. Calculates accuracy scores
    4. Returns performance comparison
    
    Parameters:
    -----------
    X_train : array-like, shape (n_samples, n_features)
        Training feature matrix (preprocessed)
    y_train : array-like, shape (n_samples,)
        Training target labels
    X_test : array-like, shape (n_samples, n_features)  
        Test feature matrix (preprocessed)
    y_test : array-like, shape (n_samples,)
        Test target labels
    models : dict
        Dictionary of model_name: model_object pairs
        
    Returns:
    --------
    evaluation : dict
        Dictionary of model_name: accuracy_score pairs
    """
    evaluation = {}
    
    print("🔄 Training and evaluating models...")
    
    for i in range(len(models)):
        # Get current model
        model = list(models.values())[i]
        model_name = list(models.keys())[i]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate accuracy
        model_score = accuracy_score(y_test, y_pred)
        
        # Store result
        evaluation[model_name] = model_score
        
        print(f"   ✅ {model_name}: {model_score:.4f}")
    
    return evaluation

print("✅ MODEL EVALUATION FUNCTION CREATED!")
print(f"\n🔧 FUNCTION CAPABILITIES:")
print(f"   • Trains multiple models automatically")
print(f"   • Standardized evaluation process")
print(f"   • Calculates accuracy for each model")
print(f"   • Returns organized results dictionary")
print(f"   • Handles any number of models")

print(f"\n💡 USAGE WORKFLOW:")
print(f"   1. Pass preprocessed training/test data")
print(f"   2. Function trains each model") 
print(f"   3. Function tests each model")
print(f"   4. Function returns accuracy comparison")
print(f"   5. Easy to identify best performing model!")

print(f"\n🚀 READY TO EVALUATE MODELS ON PREPROCESSED DATA!")

📏 CREATING MODEL EVALUATION FUNCTION:
✅ MODEL EVALUATION FUNCTION CREATED!

🔧 FUNCTION CAPABILITIES:
   • Trains multiple models automatically
   • Standardized evaluation process
   • Calculates accuracy for each model
   • Returns organized results dictionary
   • Handles any number of models

💡 USAGE WORKFLOW:
   1. Pass preprocessed training/test data
   2. Function trains each model
   3. Function tests each model
   4. Function returns accuracy comparison
   5. Easy to identify best performing model!

🚀 READY TO EVALUATE MODELS ON PREPROCESSED DATA!


In [23]:
# 🚀 TRAIN AND EVALUATE MODELS WITH PREPROCESSED DATA
# ==================================================

# Use the TRANSFORMED data for model training and evaluation
print("🚀 Training models on preprocessed data...")
print("=" * 50)

evaluation_results = model_train_eval(
    X_train_transformed,  # Use transformed training data (NOT X_train)
    y_train, 
    X_test_transformed,   # Use transformed test data (NOT X_test)
    y_test, 
    models
)

print("📊 MODEL PERFORMANCE RESULTS:")
print("=" * 40)
for model_name, accuracy in evaluation_results.items():
    print(f"{model_name:<25}: {accuracy:.4f} ({accuracy*100:.2f}%)")

print(f"\n🎯 Best performing model: {max(evaluation_results, key=evaluation_results.get)}")
print(f"🏆 Best accuracy: {max(evaluation_results.values()):.4f} ({max(evaluation_results.values())*100:.2f}%)")

print(f"\n💡 Success! Models trained on properly preprocessed data:")
print(f"   ✅ No more 'string to float' conversion errors")
print(f"   ✅ Categorical variables properly encoded as numbers")
print(f"   ✅ Numerical features properly scaled")
print(f"   ✅ All features ready for machine learning algorithms")

🚀 Training models on preprocessed data...
🔄 Training and evaluating models...
   ✅ Support Vector Classifier: 0.9592
   ✅ Decision Tree Classifier: 0.9796
📊 MODEL PERFORMANCE RESULTS:
Support Vector Classifier: 0.9592 (95.92%)
Decision Tree Classifier : 0.9796 (97.96%)

🎯 Best performing model: Decision Tree Classifier
🏆 Best accuracy: 0.9796 (97.96%)

💡 Success! Models trained on properly preprocessed data:
   ✅ No more 'string to float' conversion errors
   ✅ Categorical variables properly encoded as numbers
   ✅ Numerical features properly scaled
   ✅ All features ready for machine learning algorithms


## Notebook Summary

### What We Accomplished
1. **Data Exploration**: Loaded and analyzed the Tips dataset (244 samples, 7 features)
2. **Target Encoding**: Converted categorical time labels to numerical values
3. **Data Splitting**: Created stratified train-test split (80/20) maintaining class balance
4. **Missing Value Analysis**: Confirmed no missing values in dataset
5. **Pipeline Construction**: Built separate preprocessing pipelines for numerical and categorical features
6. **Pipeline Integration**: Combined pipelines using ColumnTransformer
7. **Data Transformation**: Applied preprocessing to both training and test sets
8. **Model Training**: Trained Support Vector Classifier and Decision Tree
9. **Hyperparameter Optimization**: Used RandomizedSearchCV to optimize Random Forest
10. **Performance Evaluation**: Compared all models with standardized metrics

### Key Results
- **Support Vector Classifier**: {evaluation_results['Support Vector Classifier']:.4f} accuracy
- **Decision Tree Classifier**: {evaluation_results['Decision Tree Classifier']:.4f} accuracy  
- **Random Forest (Optimized)**: Cross-validated and test set evaluation
- **Feature Expansion**: Original 6 features expanded after one-hot encoding
- **Data Quality**: Clean dataset with no missing values

### Technical Concepts Demonstrated
1. **Preprocessing Importance**: Categorical data conversion essential for ML algorithms
2. **Pipeline Benefits**: Automated, reproducible preprocessing prevents data leakage
3. **ColumnTransformer**: Different preprocessing for different feature types
4. **Hyperparameter Optimization**: RandomizedSearchCV for efficient parameter search
5. **Cross-Validation**: More robust performance estimation than single splits
6. **Model Comparison**: Systematic evaluation across multiple algorithms

### RandomizedSearchCV Key Points
- **Efficiency**: Samples parameter combinations instead of exhaustive search
- **Cross-Validation**: Uses k-fold CV for robust parameter evaluation  
- **Randomization**: Often finds good solutions faster than grid search
- **Scalability**: Better for large parameter spaces than GridSearchCV

### Next Steps for Further Analysis
- **Feature Engineering**: Create interaction terms or polynomial features
- **Additional Algorithms**: Try XGBoost, LightGBM, or other ensemble methods
- **Feature Selection**: Identify most important features for prediction
- **Model Interpretability**: Analyze feature importance and decision boundaries
- **Cross-Validation Strategies**: Experiment with different CV schemes

### Pipeline Architecture Benefits
- **Reproducibility**: Consistent preprocessing across train/test splits
- **Maintainability**: Easy to modify and extend preprocessing steps
- **Scalability**: Handles datasets of varying sizes
- **Flexibility**: Different preprocessing for different feature types
- **Error Prevention**: Automated process reduces manual preprocessing errors

---

## Additional Resources
- **Scikit-learn User Guide**: [Preprocessing Data](https://scikit-learn.org/stable/modules/preprocessing.html)
- **Pipeline Documentation**: [ML Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
- **RandomizedSearchCV**: [Hyperparameter Optimization](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [ ]:
# Import Random Forest Classifier
# Random Forest is an ensemble method that combines multiple decision trees
# It reduces overfitting and often provides better performance than single trees
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Initialize Random Forest Classifier
# Create a Random Forest instance with default parameters
rf = RandomForestClassifier()

# Display the classifier with its default parameters
print("Random Forest Classifier initialized with default parameters:")
rf

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [ ]:
# Examine Transformed Training Data
# Let's check the structure of our preprocessed training data
print("Transformed training data shape:", X_train_transformed.shape)
print("This data is now ready for machine learning algorithms.")
print("All categorical variables have been converted to numerical features.")

# Display first few rows (as array since it's transformed)
print("\nFirst 5 samples of transformed data:")
print(X_train_transformed[:5])

array([[ 0.12988758, -0.03485569, -0.58939884, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.6711499 ,  1.37968343,  1.53898586, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.05227791, -0.03485569,  1.53898586, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.42977647, -0.57238056,  0.47479351, ...,  1.        ,
         0.        ,  0.        ],
       [-1.0823535 , -0.86236108, -1.65359119, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.1027411 , -0.74212526,  0.47479351, ...,  1.        ,
         0.        ,  0.        ]], shape=(195, 11))

In [ ]:
# Define Hyperparameter Grid for Random Forest
# RandomizedSearchCV will randomly sample from these parameter distributions
# This is more efficient than GridSearchCV when dealing with many parameters

from sklearn.model_selection import RandomizedSearchCV

# Parameter distributions to sample from
params = {
    'max_depth': [1, 2, 3, 5, 10, None],        # Maximum depth of trees
    'n_estimators': [50, 100, 200, 300],        # Number of trees in forest
    'criterion': ['gini', 'entropy']            # Splitting criterion
}

print("Hyperparameter search space defined:")
print("- max_depth: Controls tree depth (None = unlimited)")
print("- n_estimators: Number of trees in the forest")
print("- criterion: Measure of impurity for splits")

In [ ]:
# Display Parameter Grid
# Show the complete parameter space that will be searched
print("Complete parameter grid for RandomizedSearchCV:")
for param, values in params.items():
    print(f"  {param}: {values}")
    
print(f"\nTotal possible combinations: {len(params['max_depth']) * len(params['n_estimators']) * len(params['criterion'])}")
print("RandomizedSearchCV will sample a subset of these combinations.")

{'max_depth': [1, 2, 3, 5, 10, None],
 'n_estimators': [50, 100, 200, 300],
 'criterion': ['gini', 'entropy']}

In [ ]:
# Initialize RandomizedSearchCV
# RandomizedSearchCV randomly samples parameter combinations instead of trying all
# This is much faster than GridSearchCV while often finding equally good results

clf = RandomizedSearchCV(
    rf,                          # The estimator to optimize
    param_distributions=params,  # Parameter space to search
    cv=5,                       # 5-fold cross-validation
    verbose=3,                  # Print progress information
    scoring='accuracy'          # Optimization metric
)

print("RandomizedSearchCV initialized with:")
print("- 5-fold cross-validation")
print("- Accuracy as the scoring metric")
print("- Verbose output to track progress")
print("\nRandomizedSearchCV will:")
print("1. Randomly sample parameter combinations")
print("2. Train and validate each combination using 5-fold CV")
print("3. Select the combination with highest average CV score")

In [ ]:
# Perform Hyperparameter Optimization
print("Starting RandomizedSearchCV...")
print("This will take a moment as it trains multiple models with different parameters.")
print("=" * 60)

# Fit RandomizedSearchCV on the transformed training data
clf.fit(X_train_transformed, y_train)

print("Hyperparameter optimization completed!")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=1, n_estimators=300;, score=0.821 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=1, n_estimators=300;, score=1.000 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=1, n_estimators=300;, score=1.000 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=1, n_estimators=300;, score=0.821 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=1, n_estimators=300;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.974 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.949 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.923 total time=   0.2s
[CV 

,estimator,RandomForestClassifier()
,param_distributions,"{'criterion': ['gini', 'entropy'], 'max_depth': [1, 2, ...], 'n_estimators': [50, 100, ...]}"
,n_iter,10
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,3
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [ ]:
# Best Parameters Found
# Display the optimal parameter combination found by RandomizedSearchCV
print("Best parameters found by RandomizedSearchCV:")
print("=" * 45)

best_params = clf.best_params_
for param, value in best_params.items():
    print(f"  {param}: {value}")
    
print(f"\nThese parameters produced the highest cross-validation accuracy.")
best_params

{'n_estimators': 200, 'max_depth': 2, 'criterion': 'entropy'}

In [ ]:
# Best Cross-Validation Score
# This is the average accuracy across all 5 folds using the best parameters
print("Best cross-validation score:")
print("=" * 30)

best_score = clf.best_score_
print(f"CV Accuracy: {best_score:.4f} ({best_score*100:.2f}%)")
print(f"\nThis score represents the average performance across 5 validation folds.")
print(f"It provides a more robust estimate than a single train-test split.")

best_score

np.float64(0.9692307692307693)

In [ ]:
# Evaluate Optimized Random Forest on Test Set
print("Evaluating optimized Random Forest on test data:")
print("=" * 50)

# Get the best estimator (already trained with best parameters)
best_rf = clf.best_estimator_

# Make predictions on test set
y_pred_rf = best_rf.predict(X_test_transformed)

# Calculate test accuracy
from sklearn.metrics import accuracy_score, classification_report
test_accuracy = accuracy_score(y_test, y_pred_rf)

print(f"Test Set Results:")
print(f"Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print()

# Compare with previous models
print("Model Performance Comparison:")
print("-" * 40)
print(f"Support Vector Classifier: {evaluation_results['Support Vector Classifier']:.4f}")
print(f"Decision Tree Classifier:  {evaluation_results['Decision Tree Classifier']:.4f}")
print(f"Random Forest (Optimized): {test_accuracy:.4f}")
print()

# Determine best performing model
all_results = {**evaluation_results, 'Random Forest (Optimized)': test_accuracy}
best_model = max(all_results, key=all_results.get)
best_accuracy = max(all_results.values())

print(f"Best performing model: {best_model}")
print(f"Best accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

# Show detailed classification report
print(f"\nDetailed Classification Report for Random Forest:")
print("=" * 50)
target_names = ['Dinner', 'Lunch']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

## Out-of-Bag (OOB) Score

Out-of-Bag scoring is a unique feature of Random Forest that provides an internal validation mechanism without needing a separate validation set. This technique leverages the bootstrap sampling process inherent in Random Forest construction.

### How OOB Works:
1. **Bootstrap Sampling**: Each tree in the forest is trained on a bootstrap sample (random sampling with replacement)
2. **OOB Data**: For each bootstrap sample, approximately 37% of the original data is left out (not selected)
3. **Internal Validation**: These left-out samples serve as a test set for that specific tree
4. **Aggregated Score**: The OOB score aggregates predictions across all trees using their respective OOB data

### Key Benefits:
- **No Need for Validation Split**: Uses the training data more efficiently
- **Unbiased Estimate**: Provides honest performance estimate without data leakage
- **Computational Efficiency**: No additional model fitting required
- **Model Selection**: Useful for comparing different Random Forest configurations

In [ ]:
# Demonstrate Out-of-Bag (OOB) Score
# OOB score provides internal validation without needing a separate validation set

# Import required libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

print("Demonstrating Out-of-Bag (OOB) Score:")
print("=" * 40)

# Create a synthetic dataset for demonstration
# This shows OOB scoring on a controlled dataset
X, y = make_classification(
    n_samples=1000,      # 1000 samples
    n_features=20,       # 20 features
    n_classes=2,         # Binary classification
    random_state=42      # Reproducible results
)

print(f"Synthetic dataset created: {X.shape[0]} samples, {X.shape[1]} features")

# Initialize Random Forest with OOB scoring enabled
rf_classifier = RandomForestClassifier(
    n_estimators=100,    # 100 trees in the forest
    oob_score=True,      # Enable OOB score calculation
    random_state=42      # Reproducible results
)

print("Training Random Forest with OOB scoring enabled...")

# Fit the model
rf_classifier.fit(X, y)

# Extract OOB score
oob_score = rf_classifier.oob_score_

print(f"\nOut-of-Bag Score: {oob_score:.4f} ({oob_score*100:.2f}%)")
print(f"\nWhat this means:")
print(f"  • Internal validation accuracy without separate test set")
print(f"  • Based on ~37% of data left out during bootstrap sampling")
print(f"  • Unbiased estimate of model performance")
print(f"  • No additional data splitting required")

oob_score

0.895


In [ ]:
## Classification vs Regression: Understanding the Fundamental Difference

### Problem Type Identification
The choice between classification and regression depends entirely on your **target variable type**:

#### Classification Problems
- **Target**: Discrete categories or classes
- **Goal**: Predict which category/class an instance belongs to
- **Output**: Class labels or probabilities
- **Examples**: 
  - Email spam detection (Spam/Not Spam)
  - Medical diagnosis (Disease/Healthy)
  - Customer churn (Will churn/Won't churn)
  - Image recognition (Cat/Dog/Bird)

#### Regression Problems
- **Target**: Continuous numerical values
- **Goal**: Predict a numerical value
- **Output**: Real numbers
- **Examples**:
  - House price prediction ($150,000, $200,000, etc.)
  - Stock price forecasting (continuous values)
  - Temperature prediction (degrees)
  - Sales volume forecasting (units sold)

### Key Technical Differences

| Aspect | Classification | Regression |
|--------|---------------|------------|
| **Target Variable** | Categorical (discrete) | Numerical (continuous) |
| **Output Type** | Class labels/probabilities | Real numbers |
| **Evaluation Metrics** | Accuracy, Precision, Recall, F1-score | MSE, RMSE, MAE, R² |
| **Loss Functions** | Cross-entropy, Hinge loss | Mean Squared Error, Absolute Error |
| **Decision Boundary** | Separates classes | Fits continuous curve/surface |

### Algorithm Variations

Most machine learning algorithms have both classifier and regressor versions:

#### Random Forest Example:
- **RandomForestClassifier**: For categorical targets
- **RandomForestRegressor**: For continuous targets

#### Support Vector Machines:
- **SVC (Support Vector Classifier)**: For classification
- **SVR (Support Vector Regressor)**: For regression

#### Decision Trees:
- **DecisionTreeClassifier**: Splits based on class purity
- **DecisionTreeRegressor**: Splits to minimize variance

In [ ]:
# Practical Example: Classification vs Regression with Tips Dataset
print("Classification vs Regression: Practical Demonstration")
print("=" * 55)

# CLASSIFICATION EXAMPLE: Predict meal time (our current problem)
print("1. CLASSIFICATION EXAMPLE:")
print("   Target: 'time' (Lunch=1, Dinner=0)")
print("   Problem: Predict categorical meal time")
print("   Algorithms: RandomForestClassifier, SVC, DecisionTreeClassifier")
print("   Evaluation: Accuracy, Precision, Recall, F1-score")
print()

# Show current classification results
print("   Our Classification Results:")
print(f"     • Support Vector Classifier: {evaluation_results['Support Vector Classifier']:.4f}")
print(f"     • Decision Tree Classifier:  {evaluation_results['Decision Tree Classifier']:.4f}")
print(f"     • Random Forest (Optimized): {test_accuracy:.4f}")
print()

# REGRESSION EXAMPLE: Predict tip amount
print("2. REGRESSION EXAMPLE:")
print("   Target: 'tip' (continuous dollar amounts)")
print("   Problem: Predict numerical tip value")
print("   Algorithms: RandomForestRegressor, SVR, DecisionTreeRegressor")
print("   Evaluation: MSE, RMSE, MAE, R²")
print()

# Demonstrate regression on tip prediction
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Prepare regression data (predict tip amount)
# Use all features except 'tip' to predict 'tip'
df_reg = sns.load_dataset("tips")
X_reg = df_reg.drop('tip', axis=1)
y_reg = df_reg['tip']  # Continuous target

# Apply same preprocessing (but different target)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=1
)

# Use the same preprocessor (excluding tip column)
X_train_reg_transformed = preprocessor.fit_transform(X_train_reg)
X_test_reg_transformed = preprocessor.transform(X_test_reg)

# Train regression model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train_reg_transformed, y_train_reg)

# Make predictions
y_pred_reg = rf_regressor.predict(X_test_reg_transformed)

# Calculate regression metrics
mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print("   Our Regression Results:")
print(f"     • Mean Squared Error (MSE):  {mse:.4f}")
print(f"     • Root Mean Squared Error:   {rmse:.4f}")
print(f"     • Mean Absolute Error (MAE): {mae:.4f}")
print(f"     • R² Score:                  {r2:.4f}")
print()

print("KEY DIFFERENCES DEMONSTRATED:")
print("   Classification Output: Discrete categories (0 or 1)")
print("   Regression Output:     Continuous values (tip amounts)")
print("   Classification Metric: Accuracy (% correct predictions)")
print("   Regression Metric:     RMSE (average prediction error in dollars)")

In [ ]:
# Decision Guide: When to Use Classification vs Regression
print("DECISION GUIDE: Choosing Between Classification and Regression")
print("=" * 60)
print()

# Create a decision framework
print("STEP 1: Examine Your Target Variable")
print("-" * 35)
print("Ask yourself: 'What am I trying to predict?'")
print()

# Classification indicators
print("✓ Use CLASSIFICATION when:")
print("  • Target has distinct categories (Yes/No, High/Medium/Low)")
print("  • Output should be a class label or probability")
print("  • Examples: Spam detection, disease diagnosis, customer segments")
print("  • Evaluation focuses on correct classifications")
print()

# Regression indicators
print("✓ Use REGRESSION when:")
print("  • Target is a continuous numerical value")
print("  • Output should be a specific number")
print("  • Examples: Price prediction, temperature forecasting, sales volume")
print("  • Evaluation focuses on prediction accuracy (how close to actual value)")
print()

# Edge cases
print("STEP 2: Handle Edge Cases")
print("-" * 25)
print("• Ordinal Variables (Low/Medium/High): Can be treated as either")
print("  - Classification: Preserve category distinctions")
print("  - Regression: Convert to numbers (1,2,3) if order matters")
print()
print("• Binned Continuous Variables: Often better as regression")
print("  - Example: Age groups → Use actual age for better precision")
print()

# Practical tips
print("STEP 3: Practical Considerations")
print("-" * 31)
print("• Business Context: How will predictions be used?")
print("• Data Quality: Are class boundaries clear?")
print("• Interpretability: Do stakeholders need probabilities or exact values?")
print("• Evaluation: What matters more - being right about the category or close to the exact value?")
print()

# Summary
print("QUICK REFERENCE:")
print("Classification → Categories/Classes → 'Which type?'")
print("Regression    → Numbers/Values    → 'How much?'")
print()
print("Remember: The problem type determines the algorithm choice!")
print("Same preprocessing pipeline can often be used for both approaches.")